In [1]:
from dotenv import load_dotenv
import os, json
import requests

load_dotenv()
api_key = os.getenv("API_KEY", None)
project_id = os.getenv("PROJECT_ID", None)
ibm_cloud_iam_url = os.getenv("IAM_IBM_CLOUD_URL", None)

creds = {
    "url"    : "https://us-south.ml.cloud.ibm.com",
    "apikey" : api_key
}

params = {
    "decoding_method":"greedy",
    "max_new_tokens":3000,
    "min_new_tokens":1,
    # "temperature":0.1,
    "top_k":50,
    "top_p":1,
     "repetition_penalty": 1.1,
    "stop_sequences":["```","<|user|>","<|assistant|>"],
}

In [2]:
# Prepare the payload and headers
payload = {
    "grant_type": "urn:ibm:params:oauth:grant-type:apikey",
    "apikey": api_key
}
headers = {
    'Content-Type': "application/x-www-form-urlencoded"
}

# Make a POST request while ignoring SSL certificate verification
try:
    response = requests.post(f"https://{ibm_cloud_iam_url}/identity/token", data=payload, headers=headers, verify=False)
    
    # Check if the request was successful
    response.raise_for_status()

    # Parse the JSON response
    decoded_json = response.json()
    access_token = decoded_json["access_token"]
    # print(f"Access Token: {access_token}")
except requests.exceptions.RequestException as e:
    print(f"An error occurred: {e}")

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'iam.cloud.ibm.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [3]:
import requests

GRANITE_3B_CODE_INSTRUCT = "ibm/granite-3b-code-instruct"
GRANITE_8B_CODE_INSTRUCT = "ibm/granite-8b-code-instruct"
GRANITE_20B_CODE_INSTRUCT = "ibm/granite-20b-code-instruct"
GRANITE_34B_CODE_INSTRUCT = "ibm/granite-34b-code-instruct"
GRANITE_3_8B_INSTRUCT = "ibm/granite-3-8b-instruct"

def watsonx_code_gen(prompt,model_id):

    url = "https://us-south.ml.cloud.ibm.com/ml/v1/text/generation_stream?version=2023-05-29"

    body = {
        "input": prompt,
        "parameters": params,
        "model_id": model_id,
        "project_id": project_id
    }

    headers = {
        "Accept": "application/json",
        "Content-Type": "application/json",
        "Authorization": f"Bearer {access_token}"
    }

    response = requests.post(
        url,
        headers=headers,
        json=body,
        stream=True
    )

    if response.status_code != 200:
        raise Exception("Non-200 response: " + str(response.text))

    answer = ""
    # Stream the response
    for line in response.iter_lines():
        if line:  # Ensure the line is not empty
            decoded_line = line.decode("utf-8").strip()
            
            # Check if the line starts with "data: "
            if decoded_line.startswith("data: "):
                json_data = decoded_line[len("data: "):]  # Remove the "data: " prefix
                
                try:
                    # Attempt to load the JSON data
                    data = json.loads(json_data)
                    generated_text = data.get("results", "")
                    answer += generated_text[0]["generated_text"]
                    # print(generated_text[0]["generated_text"],end="")  # Print or process the generated text as needed
                except json.JSONDecodeError:
                    print("Failed to decode JSON:", json_data)
                except Exception as e:
                    print("An error occurred:", e)

    return answer


In [4]:
from ibm_watsonx_ai.foundation_models import ModelInference
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams
import os
from dotenv import load_dotenv


load_dotenv()
api_key = os.getenv("API_KEY", None)
project_id = os.getenv("PROJECT_ID", None)

creds = {
    "url"    : "https://us-south.ml.cloud.ibm.com",
    "apikey" : api_key
}

params = {
    GenParams.DECODING_METHOD:"greedy",
    GenParams.MAX_NEW_TOKENS:4000,
    GenParams.MIN_NEW_TOKENS:1,
    # "temperature":0.1,
    GenParams.TOP_K:50,
    GenParams.TOP_P:1,
    GenParams.STOP_SEQUENCES:["```","[/INST]","<|user|>","<|endoftext|>","<|assistant|>"],
}

model = ModelInference(model_id="ibm/granite-3-8b-instruct", credentials=creds, params=params, project_id=project_id)

In [5]:
model_id = GRANITE_3_8B_INSTRUCT
language = "c"
documentframework = "doxygen"
testframework = "cunit"

In [6]:
prompt = f"""<|user|>
Please generate a sample {language} code:
-give more detail and complex logic.
-generate the code only.
-dont generate explanation.
<|assistant|>
```{language}
"""

sample_code = model.generate_text(prompt).replace("```","")
print(sample_code)

#include <stdio.h>
#include <stdlib.h>
#include <string.h>

typedef struct Node {
    int data;
    struct Node* next;
} Node;

Node* createNode(int data) {
    Node* newNode = (Node*)malloc(sizeof(Node));
    newNode->data = data;
    newNode->next = NULL;
    return newNode;
}

void insertNode(Node** head, int data) {
    Node* newNode = createNode(data);
    if (*head == NULL) {
        *head = newNode;
        return;
    }
    Node* temp = *head;
    while (temp->next != NULL) {
        temp = temp->next;
    }
    temp->next = newNode;
}

void printList(Node* head) {
    while (head != NULL) {
        printf("%d ", head->data);
        head = head->next;
    }
    printf("\n");
}

int main() {
    Node* head = NULL;
    insertNode(&head, 1);
    insertNode(&head, 2);
    insertNode(&head, 3);
    insertNode(&head, 4);
    insertNode(&head, 5);
    printList(head);
    return 0;
}



In [7]:
sample_code = """#include <stdio.h>
#include <stdlib.h>
#include <string.h>

typedef struct Node {
    int data;
    struct Node* next;
} Node;

Node* createNode(int data) {
    Node* newNode = (Node*)malloc(sizeof(Node));
    newNode->data = data;
    newNode->next = NULL;
    return newNode;
}

void insertNode(Node** head, int data) {
    Node* newNode = createNode(data);
    if (*head == NULL) {
        *head = newNode;
        return;
    }
    Node* temp = *head;
    while (temp->next != NULL) {
        temp = temp->next;
    }
    temp->next = newNode;
}

void printList(Node* head) {
    while (head != NULL) {
        printf("%d ", head->data);
        head = head->next;
    }
    printf("\n");
}

int main() {
    Node* head = NULL;
    insertNode(&head, 1);
    insertNode(&head, 2);
    insertNode(&head, 3);
    insertNode(&head, 4);
    insertNode(&head, 5);
    printList(head);
    return 0;
}
"""

bad_code = """#include <stdio.h>
#include <stdlib.h>
#include <string.h>

typedef struct Node {
    int data;
    struct Node* next;
} Node;

Node* createNode(int data) {
    Node* newNode = (Node*)malloc(sizeof(Node));
    newNode->data 
    return newNode;
}

void insertNode(Node** head, int data) {
    Node* newNode = createNode(data);
    if (*hea
    }
    Node* temp = *head;
    while (temp->next != NULL) {
        temp = temp->next;
    }
    temp->next = newNode;
}

void printList(Node* head) {
    while (head != NUL
    }
    printf("\n");
}

int main() {
    Node* head = NULL;
    insertNode(&head, 1);
    insertNode(&head, 2);
    insertNode(&head, 3);
    insertNode(&head, 4);
    insertNode(&head, 5);
    printList(head);
    return 0;
}

"""

gnerate explanation

In [8]:
prompt = f"""<|user|>
Please provide a detailed explanation for the following {language} code:
-ensure have new line after every full stop.

###
code: `{sample_code}`
<|assistant|>
explanation:
"""

answer = model.generate_text(prompt)
print(answer)


This C code is a simple implementation of a singly linked list. Here's a breakdown of the code:

1. The code starts by including the necessary libraries: `stdio.h` for input/output operations, `stdlib.h` for memory management functions like `malloc` and `free`, and `string.h` for string manipulation functions.

2. A structure `Node` is defined to represent a node in the linked list. Each node contains an integer `data` and a pointer `next` to the next node in the list.

3. The `createNode` function is defined to create a new node with the given data and return a pointer to it.

4. The `insertNode` function is defined to insert a new node at the end of the linked list. It takes a pointer to the head of the list and the data for the new node as arguments. If the list is empty, it sets the head to the new node. Otherwise, it traverses the list to the end and appends the new node.

5. The `printList` function is defined to print the data of all nodes in the linked list. It takes the head 

generate document

In [9]:
prompt = f"""<|user|>
generate {documentframework} document for the {language} provided.
-generate the {language} code only.
{language}: `{sample_code}`
<|assistant|>
```{language}"""

answer = model.generate_text(prompt).replace("```","")
print(answer)


/**
 * @file linkedlist.c
 * @brief A simple linked list implementation in C.
 *
 * This file contains the implementation of a singly linked list in C.
 * The linked list is implemented using a struct called Node, which contains
 * an integer data field and a pointer to the next Node in the list.
 *
 * @author Your Name
 * @date 2023-03-15
 */

#include <stdio.h>
#include <stdlib.h>
#include <string.h>

/**
 * @brief A struct representing a Node in the linked list.
 */
typedef struct Node {
    int data;
    struct Node* next;
} Node;

/**
 * @brief Creates a new Node with the given data and returns a pointer to it.
 *
 * @param data The data to store in the new Node.
 * @return A pointer to the new Node.
 */
Node* createNode(int data) {
    Node* newNode = (Node*)malloc(sizeof(Node));
    newNode->data = data;
    newNode->next = NULL;
    return newNode;
}

/**
 * @brief Inserts a new Node with the given data at the end of the linked list.
 *
 * @param head A pointer to the head of 

score, review and improve

In [11]:
prompt = f"""<|user|>
you are {language} expert. you need to review the following {language} code. 
-please understand the {language} code provided in detail.
-please review the code in completeness, logic, performance and quality.
-give a score out of 10 for the quality of code.
-please provide constructive suggestion in detail.
-generate the review and suggestion only.
{language} code: `{sample_code}`
-DONT generate the improve code directly.
-dont generate code.
<|assistant|>
score and review:"""

suggestion = model.generate_text(prompt)
print(suggestion)



The provided C code is a simple implementation of a singly linked list. It defines a `Node` struct to hold an integer value and a pointer to the next node. The `createNode` function creates a new node with the given data, and the `insertNode` function inserts a new node at the end of the list. The `printList` function prints the data of all nodes in the list.

The code is complete and functional, but it could be improved in several ways:

1. Error handling: The code does not check for errors when allocating memory with `malloc`. If `malloc` fails to allocate memory, it returns `NULL`, and the program will continue to run with a `NULL` pointer, which can lead to unexpected behavior.
2. Code organization: The code could be made more object-oriented by defining a `LinkedList` struct that contains a pointer to the head node. This would allow for easier manipulation of the list as a whole, rather than just individual nodes.
3. Code quality: The code could be made more readable by adding c

In [18]:
prompt = f"""<|user|>
improve the {language} provided base on the review provided.
-understand the review suggestion in detail.
-adopt the review suggestion if applicable.
-dont generate explanation.
###
original {language} code: `{sample_code}`
review suggestion: `{suggestion}`
<|assistant|>
improved {language}
```{language}"""

answer = model.generate_text(prompt).replace("```","")
print(answer)


#include <iostream>
#include <vector>
#include <algorithm>

class Person {
public:
    Person(const std::string& name, int age) : name(name), age(age) {}

    const std::string& getName() const { return name; }
    int getAge() const { return age; }

    bool operator<(const Person& other) const {
        return age < other.age;
    }

private:
    std::string name;
    int age;
};

int main() {
    std::vector<Person> people = {
        {"Alice", 30},
        {"Bob", 25},
        {"Charlie", 35},
        {"Diana", 28},
        {"Eve", 40}
    };

    std::sort(people.begin(), people.end());

    for (const auto& person : people) {
        std::cout << person.getName() << ": " << person.getAge() << std::endl;
    }

    return 0;
}



fix bad code

In [19]:
prompt = f"""<|user|>
fix the {language} provided.
-understand the review suggestion in detail.
-identify if any problem.
-fix the problem if any.
-add explanation as inline comment.

###
{language}: `{bad_code}`
<|assistant|>
fixed version with inline comment at the point it fix.
```{language}"""

answer = model.generate_text(prompt).replace("```","")
print(answer)


#include <iostream>
#include <vector>

struct Person {
    std::string name;
    int age;
};

bool compareByAge(const Person& a, const Person& b) {
    return a.age < b.age;
}

int main() {
    std::vector<Person> people = {{"Alice", 30}, {"Bob", 25}, {"Charlie", 35}};
    std::sort(people.begin(), people.end(), compareByAge); // fixed here

    for (const auto& person : people) {
        std::cout << "Name: " << person.name << ", Age: " << person.age << std::endl;
    }

    return 0;
}


The issue with the original code was that the `std::sort` function was not provided with the correct range for sorting. The range should be from the beginning of the vector to the end of the vector, which is `people.begin()` and `people.end()`, respectively. The original code was missing the second argument, which caused a compilation error.

The fixed code now correctly sorts the `people` vector by age using the `compareByAge` function and then prints the name and age of each person in the sorted v

generate diagram

In [30]:
prompt = f"""<|user|>
generate diagram for the {language} code provided.
-understand the review suggestion in detail.
-use simple name for node.
-generate in mermaid.
-dont repeating.
-dont generation explanation.
{language}: `{sample_code}`
<|assistant|>
```mermaid"""

answer = model.generate_text(prompt).replace("```","")
print(answer)


graph TD
    A[main] --> B[initialize people vector]
    B --> C[define compareByAge function]
    C --> D[sort people vector by age]
    D --> E[print sorted people vector]
    E --> F[return 0]



conversion

In [31]:
target_language = "python"

prompt = f"""<|user|>
rewrite the {language} code provided to {target_language}.
-understand the code provided in detail.
-understand how to map technology and syntax to the target {target_language} langauge.
-dont generation explanation.
{language}: `{sample_code}`
<|assistant|>
```{target_language}"""

answer = model.generate_text(prompt).replace("```","")
print(answer)


class Person:
    def __init__(self, name, age):
        self.name = name
        self.age = age

def compare_by_age(a, b):
    return a.age < b.age

people = [
    Person("Alice", 30),
    Person("Bob", 25),
    Person("Charlie", 35),
    Person("Diana", 28),
    Person("Eve", 40)
]

people.sort(key=functools.cmp_to_key(compare_by_age))

for person in people:
    print(person.name, ":", person.age)



generate api definition

In [32]:
prompt = f"""<|user|>
generate a API definition in swagger for the {language} code provided.
-dont generate explanation.
{language}: `{sample_code}`
<|assistant|>
```yaml
"""

answer = model.generate_text(prompt).replace("```","")
print(answer)

openapi: 3.0.0
info:
  title: Person Sorting API
  version: 1.0.0
servers:
  - url: http://api.example.com/people
paths:
  /sort:
    post:
      summary: Sort people by age
      requestBody:
        content:
          application/json:
            schema:
              type: array
              items:
                $ref: '#/components/schemas/Person'
      responses:
        '200':
          description: Successful response
          content:
            application/json:
              schema:
                type: array
                items:
                  $ref: '#/components/schemas/Person'
components:
  schemas:
    Person:
      type: object
      properties:
        name:
          type: string
        age:
          type: integer



generate test case

In [28]:
prompt = f"""<|user|>
generate Test Cases in {testframework} for the {language} code provided.
-focus on boundary cases.
-focus on exception handling
-generate the test only.
{language}: `{sample_code}`
<|assistant|>
```{language}"""

answer = model.generate_text(prompt).replace("```","")
print(answer)


#include <iostream>
#include <vector>
#include <algorithm>
#include <stdexcept>
#include <xunit.hpp>

struct Person {
    std::string name;
    int age;
};

bool compareByAge(const Person& a, const Person& b) {
    return a.age < b.age;
}

TEST_CASE("Sort people by age", "[sort]") {
    std::vector<Person> people = {
        {"Alice", 30},
        {"Bob", 25},
        {"Charlie", 35},
        {"Diana", 28},
        {"Eve", 40}
    };

    std::sort(people.begin(), people.end(), compareByAge);

    REQUIRE(people[0].name == "Bob");
    REQUIRE(people[1].name == "Diana");
    REQUIRE(people[2].name == "Alice");
    REQUIRE(people[3].name == "Charlie");
    REQUIRE(people[4].name == "Eve");
}

TEST_CASE("Sort people by age with equal ages", "[sort]") {
    std::vector<Person> people = {
        {"Alice", 30},
        {"Bob", 30},
        {"Charlie", 35},
        {"Diana", 30},
        {"Eve", 40}
    };

    std::sort(people.begin(), people.end(), compareByAge);

    REQUIRE(people[0].na